# IMPORT DATA

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# import files
fname1 = 'C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Data/emdat_public_2021_06_24_query_uid-s5YWMa.csv' # EMDAT natural disaster dataset
EMDAT = pd.read_csv(fname1, dtype = 'string')

fname2 = 'C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Data/CPIAUCNS.csv' # FRED consumer price index dataset
inflation = pd.read_csv(fname2)

In [ ]:
# configure plot settings
#pd.set_option('display.max_rows', None)

#SMALL_SIZE = 10
#MEDIUM_SIZE = 15
#LARGE_SIZE = 20

#plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
#plt.rc('axes', titlesize=LARGE_SIZE)     # fontsize of the axes title
#plt.rc('axes', labelsize=LARGE_SIZE)    # fontsize of the x and y labels
#plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
#plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
#plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
#plt.rc('figure', titlesize=LARGE_SIZE)  # fontsize of the figure title

# CLEAN DATA

In [ ]:
# filter out irrelevant disasters
EMDAT = EMDAT[EMDAT['Disaster Group'] != 'Technological']
EMDAT = EMDAT[EMDAT['Disaster Subgroup'] != 'Extra-terrestrial']
EMDAT = EMDAT[EMDAT['Disaster Subgroup'] != 'Biological']
EMDAT = EMDAT[EMDAT['Disaster Subgroup'] != 'Complex Disasters']
EMDAT = EMDAT[EMDAT['Disaster Type'] != 'Epidemic']
EMDAT = EMDAT[EMDAT['Disaster Type'] != 'Mass movement (dry)']

# turn string values into integers
EMDAT['Year'] = [int(year) for year in EMDAT['Year']]
EMDAT['Total Damages'] = EMDAT['Total Damages'].fillna('0')
EMDAT['Total Damages'] = [int(cost) for cost in EMDAT['Total Damages']]
EMDAT['Reconstruction Costs'] = EMDAT['Reconstruction Costs'].fillna('0')
EMDAT['Reconstruction Costs'] = [int(cost) for cost in EMDAT['Reconstruction Costs']]
EMDAT['Insured Damages'] = EMDAT['Insured Damages'].fillna('0')
EMDAT['Insured Damages'] = [int(cost) for cost in EMDAT['Insured Damages']]

# take data from last 40 years as recommended by Centre de Recherche sur L’Epidemiologie des Désastres (CRED)
EMDAT = EMDAT[EMDAT['Year'] >= 1980]
EMDAT = EMDAT[EMDAT['Year'] < 2021]

# filter out irrelevant columns
columns = ['Year', 'Country', 'Disaster Group', 'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype', 'Total Damages']
data = EMDAT[columns]
# count number of disasters in each year
disaster_counts = data['Year'].value_counts().sort_index()

# get mean CPI and CPI multiplier by year
inflation = inflation.groupby('Year').mean()

# merge tables
data = data.merge(inflation, on = 'Year')

# normalize prices to 2020 prices
data['Total Damages'] = data['Total Damages'] * data['CPI Multiplier']

# set prices from K USD to B USD
data['Total Damages'] = data['Total Damages'] / 1000000

In [ ]:
data.info()

In [ ]:
data.sort_values('Total Damages', ascending = False).head()

# SPLIT  DISASTER SUBGROUPS

In [ ]:
# display unique subgroups
for subgroup in data['Disaster Subgroup'].unique():
    print(subgroup)

In [ ]:
# create new dataframes for each subgroup
climatological_data = data[data['Disaster Subgroup'] == 'Climatological']
geophysical_data = data[data['Disaster Subgroup'] == 'Geophysical']
meteorological_data = data[data['Disaster Subgroup'] == 'Meteorological']
hydrological_data = data[data['Disaster Subgroup'] == 'Hydrological']

# count number of disasters in each year for each subgroup
climatological_disaster_counts = climatological_data['Year'].value_counts().sort_index()
geophysical_disaster_counts = geophysical_data['Year'].value_counts().sort_index()
meteorological_disaster_counts = meteorological_data['Year'].value_counts().sort_index()
hydrological_disaster_counts = hydrological_data['Year'].value_counts().sort_index()

# save index of years
disaster_counts_index = disaster_counts.index
climatological_disaster_counts_index = climatological_disaster_counts.index
geophysical_disaster_counts_index = geophysical_disaster_counts.index
meteorological_disaster_counts_index = meteorological_disaster_counts.index
hydrological_disaster_counts_index = hydrological_disaster_counts.index

# PLOT DISASTER SUBGROUPS

In [ ]:
# single line plot

#plt.figure(figsize=(20,5))

plt.grid(axis = 'both')
plt.title("Natural Disasters per Year\n")
plt.xlabel("\nYear")
plt.ylabel("Number of Natural Disasters\n")

plt.plot(disaster_counts_index, disaster_counts, linewidth=2, color = 'grey', alpha = 0.9, label = 'Total')
plt.plot(climatological_disaster_counts_index, climatological_disaster_counts, linestyle = '--', color = 'darkorange', alpha = 0.9, label = 'Climatological')
plt.plot(geophysical_disaster_counts_index, geophysical_disaster_counts, linestyle = '--', color = 'brown', alpha = 0.9, label = 'Geophysical')
plt.plot(meteorological_disaster_counts_index, meteorological_disaster_counts, linestyle = '--', color = 'midnightblue', alpha = 0.9, label = 'Meteorological')
plt.plot(hydrological_disaster_counts_index, hydrological_disaster_counts, linestyle = '--', color = 'lightseagreen', alpha = 0.9, label = 'Hydrological')
lgd = plt.legend(bbox_to_anchor=(1,1), loc = 'upper left')
plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Total Natural Disasters/total figures/disastersPerYearSLP.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# multiple line plots

fig, axs = plt.subplots(5, sharex = True)
fig.suptitle("Natural Disasters per Year\n")
axs[2].set_ylabel("Number of Natural Disasters\n")
axs[4].set_xlabel("\nYear")

axs[0].plot(disaster_counts_index, disaster_counts, linewidth=2, color = 'grey', alpha = 0.9, label = 'Total')
axs[1].plot(climatological_disaster_counts_index, climatological_disaster_counts, linestyle = '--', color = 'darkorange', alpha = 0.9, label = 'Climatological')
axs[2].plot(geophysical_disaster_counts_index, geophysical_disaster_counts, linestyle = '--', color = 'brown', alpha = 0.9, label = 'Geophysical')
axs[3].plot(meteorological_disaster_counts_index, meteorological_disaster_counts, linestyle = '--', color = 'midnightblue', alpha = 0.9, label = 'Meteorological')
axs[4].plot(hydrological_disaster_counts_index, hydrological_disaster_counts, linestyle = '--', color = 'lightseagreen', alpha = 0.9, label = 'Hydrological')

plt.sca(axs[0])
plt.yticks(np.linspace(0,300,3))
plt.grid(axis = 'both')

plt.sca(axs[1])
plt.yticks(np.linspace(0,40,3))
plt.grid(axis = 'both')

plt.sca(axs[2])
plt.yticks(np.linspace(0,40,3))
plt.grid(axis = 'both')

plt.sca(axs[3])
plt.yticks(np.linspace(0,150,3))
plt.grid(axis = 'both')

plt.sca(axs[4])
plt.yticks(np.linspace(0,200,3))
plt.grid(axis = 'both')

lgd = fig.legend(bbox_to_anchor=(1,1), loc = 'upper left')
plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Total Natural Disasters/total figures/disastersPerYearMLP.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# single bar chart

bottom1 = climatological_disaster_counts
bottom2 = np.add(bottom1, geophysical_disaster_counts).tolist()
bottom3 = np.add(bottom2, meteorological_disaster_counts).tolist()

plt.grid(axis = 'y')

plt.title("Natural Disasters per Year\n")
plt.xlabel("\nYear")
plt.ylabel("Number of Natural Disasters\n")

plt.bar(climatological_disaster_counts_index, climatological_disaster_counts, edgecolor = 'black', color = 'darkorange', alpha = 0.9, label = 'Climatological')
plt.bar(geophysical_disaster_counts_index, geophysical_disaster_counts, bottom = bottom1, edgecolor = 'black', color = 'brown', alpha = 0.9, label = 'Geophysical')
plt.bar(meteorological_disaster_counts_index, meteorological_disaster_counts, bottom = bottom2, edgecolor = 'black', color = 'midnightblue', alpha = 0.9, label = 'Meteorological')
plt.bar(hydrological_disaster_counts_index, hydrological_disaster_counts, bottom = bottom3, edgecolor = 'black', color = 'lightseagreen', alpha = 0.9, label = 'Hydrological')

lgd = plt.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Total Natural Disasters/total figures/disastersPerYearSBC.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# multiple bar charts

fig, axs = plt.subplots(5, sharex = True)
fig.suptitle("Natural Disasters per Year\n")
axs[2].set_ylabel("Number of Natural Disasters\n")
axs[4].set_xlabel("\nYear")

axs[0].bar(disaster_counts_index, disaster_counts, edgecolor = 'black', color = 'grey', alpha = 0.9, label = 'Total')
axs[1].bar(climatological_disaster_counts_index, climatological_disaster_counts, edgecolor = 'grey', color = 'darkorange', alpha = 0.9, label = 'Climatological')
axs[2].bar(geophysical_disaster_counts_index, geophysical_disaster_counts, edgecolor = 'grey', color = 'brown', alpha = 0.9, label = 'Geophysical')
axs[3].bar(meteorological_disaster_counts_index, meteorological_disaster_counts, edgecolor = 'grey', color = 'midnightblue', alpha = 0.9, label = 'Meteorological')
axs[4].bar(hydrological_disaster_counts_index, hydrological_disaster_counts, edgecolor = 'grey', color = 'lightseagreen', alpha = 0.9, label = 'Hydrological')

plt.sca(axs[0])
plt.yticks(np.linspace(0,300,3))
plt.grid(axis = 'y')

plt.sca(axs[1])
plt.yticks(np.linspace(0,40,3))
plt.grid(axis = 'y')

plt.sca(axs[2])
plt.yticks(np.linspace(0,40,3))
plt.grid(axis = 'y')

plt.sca(axs[3])
plt.yticks(np.linspace(0,150,3))
plt.grid(axis = 'y')

plt.sca(axs[4])
plt.yticks(np.linspace(0,200,3))
plt.grid(axis = 'y')


lgd = fig.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Total Natural Disasters/total figures/disastersPerYearMBC.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

# BOXPLOT DISASTERS BY DECADE

In [ ]:
total_box_data = [disaster_counts[0:10], disaster_counts[10:20], disaster_counts[20:30], disaster_counts[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

x0 = [np.random.normal(1,0.04,10)]
x1 = [np.random.normal(2,0.04,10)]
x2 = [np.random.normal(3,0.04,10)]
x3 = [np.random.normal(4,0.04,10)]

plt.figure(figsize=(3,6))
plt.title("Natural Disasters per Year\n")
plt.xlabel("\nDecade")
plt.ylabel("Number of Natural Disasters\n")

plt.boxplot(total_box_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,total_box_data[0], color = 'grey', alpha = 0.5)
plt.scatter(x1,total_box_data[1], color = 'grey', alpha = 0.5)
plt.scatter(x2,total_box_data[2], color = 'grey', alpha = 0.5)
plt.scatter(x3,total_box_data[3], color = 'grey', alpha = 0.5)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Total Natural Disasters/total figures/disastersPerYearBOX.png", bbox_inches='tight')

# ANALYZE DISASTER OCCURANCES

In [ ]:
# TOTAL NATURAL DISASTER ANALYSIS

print("TOTAL NATURAL DISASTER ANALYSIS")

# total statistical data
print("\nTotal Statistical Data: ")
print(disaster_counts.describe())

# 1980s statistical data
print("\n1980s Statistical Data: ")
print(disaster_counts[0:10].describe())

# 1990s statistical data
print("\n1990s Statistical Data: ")
print(disaster_counts[10:20].describe())

# 2000s statistical data
print("\n2000s Statistical Data: ")
print(disaster_counts[20:30].describe())

# 2010s statistical data
print("\n2010s Statistical Data: ")
print(disaster_counts[30:40].describe())

print("\n")

# total statistical changes 1980s-2010s

total_mean_1980s = disaster_counts[0:10].mean()
total_mean_2010s = disaster_counts[30:40].mean()
mean_percent_change_1980_to_2020 = 100*(total_mean_2010s-total_mean_1980s)/total_mean_1980s
print("Mean Disaster Change (1980s-2010s): {:.2f}%".format(mean_percent_change_1980_to_2020))

total_max_1980s = disaster_counts[0:10].max()
total_max_2010s = disaster_counts[30:40].max()
max_percent_change_1980_to_2020 = 100*(total_max_2010s-total_max_1980s)/total_max_1980s
print("Max Disaster Change (1980s-2010s): {:.2f}%".format(max_percent_change_1980_to_2020))

total_min_1980s = disaster_counts[0:10].min()
total_min_2010s = disaster_counts[30:40].min()
min_percent_change_1980_to_2020 = 100*(total_min_2010s-total_min_1980s)/total_min_1980s
print("Min Disaster Change (1980s-2010s): {:.2f}%".format(min_percent_change_1980_to_2020))

total_std_1980s = disaster_counts[0:10].std()
total_std_2010s = disaster_counts[30:40].std()
std_percent_change_1980_to_2020 = 100*(total_std_2010s-total_std_1980s)/total_std_1980s
print("Standard Deviation of Disasters Change (1980s-2010s): {:.2f}%".format(std_percent_change_1980_to_2020))

In [ ]:
# CLIMATOLOGICAL DISASTER ANALYSIS

print("CLIMATOLOGICAL DISASTER ANALYSIS")

# climatological statistical data
print("\nTotal Climatological Statistical Data: ")
print(climatological_disaster_counts.describe())

# 1980s climatological statistical data
print("\n1980s Climatological Statistical Data: ")
print(climatological_disaster_counts[0:10].describe())

# 1990s climatological statistical data
print("\n1990s Climatological Statistical Data: ")
print(climatological_disaster_counts[10:20].describe())

# 2000s climatological statistical data
print("\n2000s Climatological Statistical Data: ")
print(climatological_disaster_counts[20:30].describe())

# 2010s climatological statistical data
print("\n2010s Climatological Statistical Data: ")
print(climatological_disaster_counts[30:40].describe())

print("\n")

# climatological statistical changes 1980s-2010s
climatological_mean_1980s = climatological_disaster_counts[0:10].mean()
climatological_mean_2010s = climatological_disaster_counts[30:40].mean()
mean_percent_change_climatological_1980_to_2020 = 100*(climatological_mean_2010s-climatological_mean_1980s)/climatological_mean_1980s
print("Mean Climatological Disaster Change (1980s-2010s): {:.2f}%".format(mean_percent_change_climatological_1980_to_2020))

climatological_max_1980s = climatological_disaster_counts[0:10].max()
climatological_max_2010s = climatological_disaster_counts[30:40].max()
max_percent_change_climatological_1980_to_2020 = 100*(climatological_max_2010s-climatological_max_1980s)/climatological_max_1980s
print("Max Climatological Disaster Change (1980s-2010s): {:.2f}%".format(max_percent_change_climatological_1980_to_2020))

climatological_min_1980s = climatological_disaster_counts[0:10].min()
climatological_min_2010s = climatological_disaster_counts[30:40].min()
min_percent_change_climatological_1980_to_2020 = 100*(climatological_min_2010s-climatological_min_1980s)/climatological_min_1980s
print("Min Climatological Disaster Change (1980s-2010s): {:.2f}%".format(min_percent_change_climatological_1980_to_2020))

climatological_std_1980s = climatological_disaster_counts[0:10].std()
climatological_std_2010s = climatological_disaster_counts[30:40].std()
std_percent_change_climatological_1980_to_2020 = 100*(climatological_std_2010s-climatological_std_1980s)/climatological_std_1980s
print("Standard Deviation of Climatological Disasters Change (1980s-2010s): {:.2f}%\n".format(std_percent_change_climatological_1980_to_2020))

In [ ]:
# GEOPHYSICAL DISASTER ANALYSIS

print("GEOPHYSICAL DISASTER ANALYSIS")

# geophysical statistical data
print("\nTotal Geophysical Statistical Data: ")
print(geophysical_disaster_counts.describe())

# 1980s geophysical statistical data
print("\n1980s Geophysical Statistical Data: ")
print(geophysical_disaster_counts[0:10].describe())

# 1990s geophysical statistical data
print("\n1990s Geophysical Statistical Data: ")
print(geophysical_disaster_counts[10:20].describe())

# 2000s geophysical statistical data
print("\n2000s Geophysical Statistical Data: ")
print(geophysical_disaster_counts[20:30].describe())

# 2010s geophysical statistical data
print("\n2010s Geophysical Statistical Data: ")
print(geophysical_disaster_counts[30:40].describe())

print("\n")

# geophysical statistical changes 1980s-2010s
geophysical_mean_1980s = geophysical_disaster_counts[0:10].mean()
geophysical_mean_2010s = geophysical_disaster_counts[30:40].mean()
mean_percent_change_geophysical_1980_to_2020 = 100*(geophysical_mean_2010s-geophysical_mean_1980s)/geophysical_mean_1980s
print("Mean Geophysical Disaster Change (1980s-2010s): {:.2f}%".format(mean_percent_change_geophysical_1980_to_2020))

geophysical_max_1980s = geophysical_disaster_counts[0:10].max()
geophysical_max_2010s = geophysical_disaster_counts[30:40].max()
max_percent_change_geophysical_1980_to_2020 = 100*(geophysical_max_2010s-geophysical_max_1980s)/geophysical_max_1980s
print("Max Geophysical Disaster Change (1980s-2010s): {:.2f}%".format(max_percent_change_geophysical_1980_to_2020))

geophysical_min_1980s = geophysical_disaster_counts[0:10].min()
geophysical_min_2010s = geophysical_disaster_counts[30:40].min()
min_percent_change_geophysical_1980_to_2020 = 100*(geophysical_min_2010s-geophysical_min_1980s)/geophysical_min_1980s
print("Min Geophysical Disaster Change (1980s-2010s): {:.2f}%".format(min_percent_change_geophysical_1980_to_2020))

geophysical_std_1980s = geophysical_disaster_counts[0:10].std()
geophysical_std_2010s = geophysical_disaster_counts[30:40].std()
std_percent_change_geophysical_1980_to_2020 = 100*(geophysical_std_2010s-geophysical_std_1980s)/geophysical_std_1980s
print("Standard Deviation of Geophysical Disasters Change (1980s-2010s): {:.2f}%\n".format(std_percent_change_geophysical_1980_to_2020))

In [ ]:
# METEOROLOGICAL DISASTER ANALYSIS

print("METEOROLOGICAL DISASTER ANALYSIS")

# meteorological statistical data
print("\nTotal Meteorological Statistical Data: ")
print(meteorological_disaster_counts.describe())

# 1980s meteorological statistical data
print("\n1980s Meteorological Statistical Data: ")
print(meteorological_disaster_counts[0:10].describe())

# 1990s meteorological statistical data
print("\n1990s Meteorological Statistical Data: ")
print(meteorological_disaster_counts[10:20].describe())

# 2000s meteorological statistical data
print("\n2000s Meteorological Statistical Data: ")
print(meteorological_disaster_counts[20:30].describe())

# 2010s meteorological statistical data
print("\n2010s Meteorological Statistical Data: ")
print(meteorological_disaster_counts[30:40].describe())

print("\n")

# meteorological statistical changes 1980s-2010s
meteorological_mean_1980s = meteorological_disaster_counts[0:10].mean()
meteorological_mean_2010s = meteorological_disaster_counts[30:40].mean()
mean_percent_change_meteorological_1980_to_2020 = 100*(meteorological_mean_2010s-meteorological_mean_1980s)/meteorological_mean_1980s
print("Mean Meteorological Disaster Change (1980s-2010s): {:.2f}%".format(mean_percent_change_meteorological_1980_to_2020))

meteorological_max_1980s = meteorological_disaster_counts[0:10].max()
meteorological_max_2010s = meteorological_disaster_counts[30:40].max()
max_percent_change_meteorological_1980_to_2020 = 100*(meteorological_max_2010s-meteorological_max_1980s)/meteorological_max_1980s
print("Max Meteorological Disaster Change (1980s-2010s): {:.2f}%".format(max_percent_change_meteorological_1980_to_2020))

meteorological_min_1980s = meteorological_disaster_counts[0:10].min()
meteorological_min_2010s = meteorological_disaster_counts[30:40].min()
min_percent_change_meteorological_1980_to_2020 = 100*(meteorological_min_2010s-meteorological_min_1980s)/meteorological_min_1980s
print("Min Meteorological Disaster Change (1980s-2010s): {:.2f}%".format(min_percent_change_meteorological_1980_to_2020))

meteorological_std_1980s = meteorological_disaster_counts[0:10].std()
meteorological_std_2010s = meteorological_disaster_counts[30:40].std()
std_percent_change_meteorological_1980_to_2020 = 100*(meteorological_std_2010s-meteorological_std_1980s)/meteorological_std_1980s
print("Standard Deviation of Meteorological Disasters Change (1980s-2010s): {:.2f}%\n".format(std_percent_change_meteorological_1980_to_2020))

In [ ]:
# HYDROLOGICAL DISASTER ANALYSIS

print("HYDROLOGICAL DISASTER ANALYSIS")

# hydrological statistical data
print("\nTotal Hydrological Statistical Data: ")
print(hydrological_disaster_counts.describe())

# 1980s hydrological statistical data
print("\n1980s Hydrological Statistical Data: ")
print(hydrological_disaster_counts[0:10].describe())

# 1990s hydrological statistical data
print("\n1990s Hydrological Statistical Data: ")
print(hydrological_disaster_counts[10:20].describe())

# 2000s hydrological statistical data
print("\n2000s Hydrological Statistical Data: ")
print(hydrological_disaster_counts[20:30].describe())

# 2010s hydrological statistical data
print("\n2010s Hydrological Statistical Data: ")
print(hydrological_disaster_counts[30:40].describe())

print("\n")

# hydrological statistical changes 1980s-2010s
hydrological_mean_1980s = hydrological_disaster_counts[0:10].mean()
hydrological_mean_2010s = hydrological_disaster_counts[30:40].mean()
mean_percent_change_hydrological_1980_to_2020 = 100*(hydrological_mean_2010s-hydrological_mean_1980s)/hydrological_mean_1980s
print("Mean Hydrological Disaster Change (1980s-2010s): {:.2f}%".format(mean_percent_change_hydrological_1980_to_2020))

hydrological_max_1980s = hydrological_disaster_counts[0:10].max()
hydrological_max_2010s = hydrological_disaster_counts[30:40].max()
max_percent_change_hydrological_1980_to_2020 = 100*(hydrological_max_2010s-hydrological_max_1980s)/hydrological_max_1980s
print("Max Hydrological Disaster Change (1980s-2010s): {:.2f}%".format(max_percent_change_hydrological_1980_to_2020))

hydrological_min_1980s = hydrological_disaster_counts[0:10].min()
hydrological_min_2010s = hydrological_disaster_counts[30:40].min()
min_percent_change_hydrological_1980_to_2020 = 100*(hydrological_min_2010s-hydrological_min_1980s)/hydrological_min_1980s
print("Min Hydrological Disaster Change (1980s-2010s): {:.2f}%".format(min_percent_change_hydrological_1980_to_2020))

hydrological_std_1980s = hydrological_disaster_counts[0:10].std()
hydrological_std_2010s = hydrological_disaster_counts[30:40].std()
std_percent_change_hydrological_1980_to_2020 = 100*(hydrological_std_2010s-hydrological_std_1980s)/hydrological_std_1980s
print("Standard Deviation of Hydrological Disasters Change (1980s-2010s): {:.2f}%\n".format(std_percent_change_hydrological_1980_to_2020))

# DISASTER SUBGROUP COSTS

In [ ]:
disaster_costs = data[['Year', 'Total Damages']]
disaster_costs = disaster_costs.groupby('Year').sum()
disaster_costs = disaster_costs.squeeze()

climatological_costs = climatological_data[['Year', 'Total Damages']]
climatological_costs = climatological_costs.groupby('Year').sum()
climatological_costs = climatological_costs.squeeze()

geophysical_costs = geophysical_data[['Year', 'Total Damages']]
geophysical_costs = geophysical_costs.groupby('Year').sum()
geophysical_costs = geophysical_costs.squeeze()

meteorological_costs = meteorological_data[['Year', 'Total Damages']]
meteorological_costs = meteorological_costs.groupby('Year').sum().groupby('Year').sum()
meteorological_costs = meteorological_costs.squeeze()

hydrological_costs = hydrological_data[['Year', 'Total Damages']]
hydrological_costs = hydrological_costs.groupby('Year').sum()
hydrological_costs = hydrological_costs.squeeze()

In [ ]:
japan_tsunami_2011_cost = 2.408585e+02
hurricane_harvey_2017_cost = 9.998350e+01
sichuan_earthquake_2008_cost = 1.018695e+02
hurricane_katrina_2005_cost = 1.651464e+02
izmit_earthquake_1999_cost = 3.097570e+01
kobe_earthquake_1995_cost = 1.692989e+02
irpinia_earthquake_1980_cost = 6.266630e+01

In [ ]:
# single line plot

plt.figure(figsize=(20,5))

plt.grid(axis = 'both')
plt.title("Natural Disaster Damages per Year\n")
plt.xlabel("\nYear")
plt.ylabel("Damages, USD Billion (2020)\n")

plt.plot(disaster_costs.index, disaster_costs, linewidth=2, color = 'grey', alpha = 0.9, label = 'Total')
plt.plot(climatological_costs.index, climatological_costs, linestyle = '--', color = 'darkorange', alpha = 0.9, label = 'Climatological')
plt.plot(geophysical_costs.index, geophysical_costs, linestyle = '--', alpha = 0.9, color = 'brown', label = 'Geophysical')
plt.plot(meteorological_costs.index, meteorological_costs, linestyle = '--', alpha = 0.9, color = 'midnightblue', label = 'Meteorological')
plt.plot(hydrological_costs.index, hydrological_costs, linestyle = '--', alpha = 0.9, color = 'lightseagreen', label = 'Hydrological')

lgd = plt.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.scatter([1980, 1995, 1999, 2005, 2008, 2011, 2017], [irpinia_earthquake_1980_cost, kobe_earthquake_1995_cost,izmit_earthquake_1999_cost,hurricane_katrina_2005_cost,sichuan_earthquake_2008_cost,japan_tsunami_2011_cost, hurricane_harvey_2017_cost], c='black', marker = "*")

plt.annotate("Irpinia Earthquake",
            xy=(1980, irpinia_earthquake_1980_cost), xycoords='data',
            xytext=(1980+1, irpinia_earthquake_1980_cost+50), textcoords='data',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            )

plt.annotate("Great Hanshin Earthquake",
            xy=(1995, kobe_earthquake_1995_cost), xycoords='data',
            xytext=(1995-5, kobe_earthquake_1995_cost+150), textcoords='data',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            )

plt.annotate("İzmit Earthquake",
            xy=(1999, izmit_earthquake_1999_cost), xycoords='data',
            xytext=(1999-2, izmit_earthquake_1999_cost+200), textcoords='data',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            )

plt.annotate("Hurricane Katrina",
            xy=(2005, hurricane_katrina_2005_cost), xycoords='data',
            xytext=(2005-4, hurricane_katrina_2005_cost+150), textcoords='data',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            )

plt.annotate("Sichuan Earthquake",
            xy=(2008, sichuan_earthquake_2008_cost), xycoords='data',
            xytext=(2008-2.5, sichuan_earthquake_2008_cost+200), textcoords='data',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            )

plt.annotate("Tōhoku Earthquake",
            xy=(2011, japan_tsunami_2011_cost), xycoords='data',
            xytext=(2011+1, japan_tsunami_2011_cost+100), textcoords='data',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            )

plt.annotate("Hurricane Harvey",
            xy=(2017, hurricane_harvey_2017_cost), xycoords='data',
            xytext=(2017+1, hurricane_harvey_2017_cost+150), textcoords='data',
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            )

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Total Natural Disasters/total figures/disasterCostsPerYearSLP.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# multiple line plots

fig, axs = plt.subplots(5, sharex = True)
fig.suptitle("Natural Disaster Damages per Year\n")
axs[2].set_ylabel("Damages, USD Billion (2020)\n")
axs[4].set_xlabel("\nYear")

axs[0].plot(disaster_costs.index, disaster_costs, linewidth=2, color = 'grey', alpha = 0.9, label = 'Total')
axs[1].plot(climatological_costs.index, climatological_costs, linestyle = '--', color = 'darkorange', alpha = 0.9, label = 'Climatological')
axs[2].plot(geophysical_costs.index, geophysical_costs, linestyle = '--', color = 'brown', alpha = 0.9, label = 'Geophysical')
axs[3].plot(meteorological_costs.index, meteorological_costs, linestyle = '--', color = 'midnightblue', alpha = 0.9, label = 'Meteorological')
axs[4].plot(hydrological_costs.index, hydrological_costs, linestyle = '--', color = 'lightseagreen', alpha = 0.9, label = 'Hydrological')

plt.sca(axs[0])
plt.yticks(np.linspace(0,300,3))
plt.grid(axis = 'both')

plt.sca(axs[1])
plt.yticks(np.linspace(0,30,3))
plt.grid(axis = 'both')

plt.sca(axs[2])
plt.yticks(np.linspace(0,200,3))
plt.grid(axis = 'both')

plt.sca(axs[3])
plt.yticks(np.linspace(0,200,3))
plt.grid(axis = 'both')

plt.sca(axs[4])
plt.yticks(np.linspace(0,60,3))
plt.grid(axis = 'both')

lgd = fig.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Total Natural Disasters/total figures/disasterCostsPerYearMLP.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# single bar chart

bottom1 = climatological_costs
bottom2 = np.add(bottom1, geophysical_costs).tolist()
bottom3 = np.add(bottom2, meteorological_costs).tolist()

plt.grid(axis = 'y')
plt.title("Natural Disaster Damages per Year\n")
plt.xlabel("\nYear")
plt.ylabel("Damages, USD Billion (2020)\n")

plt.bar(climatological_costs.index, climatological_costs, edgecolor='black', color = 'darkorange', alpha = 0.9, label = "Climatological")
plt.bar(geophysical_costs.index, geophysical_costs, bottom = bottom1, edgecolor='black', color = 'brown', alpha = 0.9, label = "Geophysical")
plt.bar(meteorological_costs.index, meteorological_costs, bottom = bottom2, edgecolor='black', color = 'midnightblue', alpha = 0.9, label = "Meteorological")
plt.bar(hydrological_costs.index, hydrological_costs, bottom = bottom3, edgecolor='black', color = 'lightseagreen', alpha = 0.9, label = "Hydrological")

lgd = plt.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Total Natural Disasters/total figures/disasterCostsPerYearSBC.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# multiple bar charts

fig, axs = plt.subplots(5, sharex = True)
fig.suptitle("Natural Disaster Damages per Year\n")
axs[2].set_ylabel("Damages, USD Billion (2020)\n")
axs[4].set_xlabel("\nYear")

axs[0].bar(disaster_costs.index, disaster_costs, color = 'grey', alpha = 0.9, edgecolor = 'black', label = 'Total')
axs[1].bar(climatological_costs.index, climatological_costs, color = 'darkorange', alpha = 0.9, edgecolor = 'grey', label = 'Climatological')
axs[2].bar(geophysical_costs.index, geophysical_costs, color = 'brown', alpha = 0.9, edgecolor = 'grey', label = 'Geophysical')
axs[3].bar(meteorological_costs.index, meteorological_costs, color = 'midnightblue', alpha = 0.9, edgecolor = 'grey', label = 'Meteorological')
axs[4].bar(hydrological_costs.index, hydrological_costs, color = 'lightseagreen', alpha = 0.9, edgecolor = 'grey', label = 'Hydrological')

plt.sca(axs[0])
plt.yticks(np.linspace(0,300,3))
plt.grid(axis = 'y')

plt.sca(axs[1])
plt.yticks(np.linspace(0,30,3))
plt.grid(axis = 'y')

plt.sca(axs[2])
plt.yticks(np.linspace(0,200,3))
plt.grid(axis = 'y')

plt.sca(axs[3])
plt.yticks(np.linspace(0,200,3))
plt.grid(axis = 'y')

plt.sca(axs[4])
plt.yticks(np.linspace(0,60,3))
plt.grid(axis = 'y')

lgd = fig.legend(bbox_to_anchor=(1,1), loc = 'upper left')

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Total Natural Disasters/total figures/disasterCostsPerYearMBC.png", bbox_extra_artists=(lgd,), bbox_inches='tight')

# BOXPLOT TOTAL DAMAGES BY DECADE

In [ ]:
total_box_cost_data = [disaster_costs[0:10], disaster_costs[10:20], disaster_costs[20:30], disaster_costs[30:40]]
box_labels = ['1980s', '1990s', '2000s', '2010s']

plt.figure(figsize=(3,6))
plt.title("Natural Disaster Damages per Year\n")
plt.xlabel("\nDecade")
plt.ylabel("Total Damages, USD Billion (2020)\n")

plt.boxplot(total_box_cost_data, labels = box_labels, whis = [0, 100])

plt.scatter(x0,total_box_cost_data[0], color = 'grey', alpha = 0.5)
plt.scatter(x1,total_box_cost_data[1], color = 'grey', alpha = 0.5)
plt.scatter(x2,total_box_cost_data[2], color = 'grey', alpha = 0.5)
plt.scatter(x3,total_box_cost_data[3], color = 'grey', alpha = 0.5)

plt.savefig("C:/Users/Felix/Documents/GitHub/naturalDisasterAnalysis/Total Natural Disasters/total figures/disasterCostsPerYearBOX.png", bbox_inches='tight')


# ANALYZE DISASTER COSTS

In [ ]:
# TOTAL NATURAL DISASTER COST ANALYSIS

print("TOTAL NATURAL DISASTER COST ANALYSIS")

# total costs statistical data
print("\nTotal Costs Statistical Data: ")
print(disaster_costs.describe())

# 1980s costs statistical data
print("\n1980s Costs Statistical Data: ")
print(disaster_costs[0:10].describe())

# 1990s costs statistical data
print("\n1990s Costs Statistical Data: ")
print(disaster_costs[10:20].describe())

# 2000s costs statistical data
print("\n2000s Costs Statistical Data: ")
print(disaster_costs[20:30].describe())

# 2010s costs statistical data
print("\n2010s Costs Statistical Data: ")
print(disaster_costs[30:40].describe())

print("\n")

# total costs statistical changes 1980s-2010s
total_mean_costs_1980s = disaster_costs[0:10].mean()
total_mean_costs_2010s = disaster_costs[30:40].mean()
mean_costs_percent_change_1980_to_2020 = 100*(total_mean_costs_2010s-total_mean_costs_1980s)/total_mean_costs_1980s
print("Mean Disaster Costs Change (1980s-2010s): {:.2f}%".format(mean_costs_percent_change_1980_to_2020))

total_max_costs_1980s = disaster_costs[0:10].max()
total_max_costs_2010s = disaster_costs[30:40].max()
max_costs_percent_change_1980_to_2020 = 100*(total_max_costs_2010s-total_max_costs_1980s)/total_max_costs_1980s
print("Max Disaster Costs Change (1980s-2010s): {:.2f}%".format(max_costs_percent_change_1980_to_2020))

total_min_costs_1980s = disaster_costs[0:10].min()
total_min_costs_2010s = disaster_costs[30:40].min()
min_costs_percent_change_1980_to_2020 = 100*(total_min_costs_2010s-total_min_costs_1980s)/total_min_costs_1980s
print("Min Disaster Costs Change (1980s-2010s): {:.2f}%".format(min_costs_percent_change_1980_to_2020))

total_std_costs_1980s = disaster_costs[0:10].std()
total_std_costs_2010s = disaster_costs[30:40].std()
std_costs_percent_change_1980_to_2020 = 100*(total_std_costs_2010s-total_std_costs_1980s)/total_std_costs_1980s
print("Standard Deviation of Disaster Costs Change (1980s-2010s): {:.2f}%\n".format(std_costs_percent_change_1980_to_2020))

In [ ]:
# CLIMATOLOGICAL DISASTER COST ANALYSIS

print("CLIMATOLOGICAL DISASTER COST ANALYSIS")

# climatological costs statistical data
print("\nTotal Climatological Costs Statistical Data: ")
print(climatological_costs.describe())

# 1980s climatological costs statistical data
print("\n1980s Climatological Costs Statistical Data: ")
print(climatological_costs[0:10].describe())

# 1990s climatological costs statistical data
print("\n1990s Climatological Costs Statistical Data: ")
print(climatological_costs[10:20].describe())

# 2000s climatological costs statistical data
print("\n2000s Climatological Costs Statistical Data: ")
print(climatological_costs[20:30].describe())

# 2010s climatological costs statistical data
print("\n2010s Climatological Costs Statistical Data: ")
print(climatological_costs[30:40].describe())

print("\n")

# climatological costs statistical changes 1980s-2010s
climatological_mean_costs_1980s = climatological_costs[0:10].mean()
climatological_mean_costs_2010s = climatological_costs[30:40].mean()
mean_costs_percent_change_climatological_1980_to_2020 = 100*(climatological_mean_costs_2010s-climatological_mean_costs_1980s)/climatological_mean_costs_1980s
print("Mean Climatological Disaster Costs Change (1980s-2010s): {:.2f}%".format(mean_costs_percent_change_climatological_1980_to_2020))

climatological_max_costs_1980s = climatological_costs[0:10].max()
climatological_max_costs_2010s = climatological_costs[30:40].max()
max_costs_percent_change_climatological_1980_to_2020 = 100*(climatological_max_costs_2010s-climatological_max_costs_1980s)/climatological_max_costs_1980s
print("Max Climatological Disaster Costs Change (1980s-2010s): {:.2f}%".format(max_costs_percent_change_climatological_1980_to_2020))

climatological_min_costs_1980s = climatological_costs[0:10].min()
climatological_min_costs_2010s = climatological_costs[30:40].min()
min_costs_percent_change_climatological_1980_to_2020 = 100*(climatological_min_costs_2010s-climatological_min_costs_1980s)/climatological_min_costs_1980s
print("Min Climatological Disaster Costs Change (1980s-2010s): {:.2f}%".format(min_costs_percent_change_climatological_1980_to_2020))

climatological_std_costs_1980s = climatological_costs[0:10].std()
climatological_std_costs_2010s = climatological_costs[30:40].std()
std_costs_percent_change_climatological_1980_to_2020 = 100*(climatological_std_costs_2010s-climatological_std_costs_1980s)/climatological_std_costs_1980s
print("Standard Deviation of Climatological Disaster Costs Change (1980s-2010s): {:.2f}%\n".format(std_costs_percent_change_climatological_1980_to_2020))

In [ ]:
# GEOPHYSICAL DISASTER COST ANALYSIS

print("GEOPHYSICAL DISASTER COST ANALYSIS")

# geophysical costs statistical data
print("\nTotal Geophysical Costs Statistical Data: ")
print(geophysical_costs.describe())

# 1980s geophysical costs statistical data
print("\n1980s Geophysical Costs Statistical Data: ")
print(geophysical_costs[0:10].describe())

# 1990s geophysical costs statistical data
print("\n1990s Geophysical Costs Statistical Data: ")
print(geophysical_costs[10:20].describe())

# 2000s geophysical costs statistical data
print("\n2000s Geophysical Costs Statistical Data: ")
print(geophysical_costs[20:30].describe())

# 2010s geophysical costs statistical data
print("\n2010s Geophysical Costs Statistical Data: ")
print(geophysical_costs[30:40].describe())

print("\n")

# geophysical costs statistical changes 1980s-2010s
geophysical_mean_costs_1980s = geophysical_costs[0:10].mean()
geophysical_mean_costs_2010s = geophysical_costs[30:40].mean()
mean_costs_percent_change_geophysical_1980_to_2020 = 100*(geophysical_mean_costs_2010s-geophysical_mean_costs_1980s)/geophysical_mean_costs_1980s
print("Mean Geophysical Disaster Costs Change (1980s-2010s): {:.2f}%".format(mean_costs_percent_change_geophysical_1980_to_2020))

geophysical_max_costs_1980s = geophysical_costs[0:10].max()
geophysical_max_costs_2010s = geophysical_costs[30:40].max()
max_costs_percent_change_geophysical_1980_to_2020 = 100*(geophysical_max_costs_2010s-geophysical_max_costs_1980s)/geophysical_max_costs_1980s
print("Max Geophysical Disaster Costs Change (1980s-2010s): {:.2f}%".format(max_costs_percent_change_geophysical_1980_to_2020))

geophysical_min_costs_1980s = geophysical_costs[0:10].min()
geophysical_min_costs_2010s = geophysical_costs[30:40].min()
min_costs_percent_change_geophysical_1980_to_2020 = 100*(geophysical_min_costs_2010s-geophysical_min_costs_1980s)/geophysical_min_costs_1980s
print("Min Geophysical Disaster Costs Change (1980s-2010s): {:.2f}%".format(min_costs_percent_change_geophysical_1980_to_2020))

geophysical_std_costs_1980s = geophysical_costs[0:10].std()
geophysical_std_costs_2010s = geophysical_costs[30:40].std()
std_costs_percent_change_geophysical_1980_to_2020 = 100*(geophysical_std_costs_2010s-geophysical_std_costs_1980s)/geophysical_std_costs_1980s
print("Standard Deviation of Geophysical Disaster Costs Change (1980s-2010s): {:.2f}%\n".format(std_costs_percent_change_geophysical_1980_to_2020))

In [ ]:
# METEOROLOGICAL DISASTER COST ANALYSIS

print("METEOROLOGICAL DISASTER COST ANALYSIS")

# meteorological costs statistical data
print("\nTotal Meteorological Costs Statistical Data: ")
print(meteorological_costs.describe())

# 1980s meteorological costs statistical data
print("\n1980s Meteorological Costs Statistical Data: ")
print(meteorological_costs[0:10].describe())

# 1990s meteorological costs statistical data
print("\n1990s Meteorological Costs Statistical Data: ")
print(meteorological_costs[10:20].describe())

# 2000s meteorological costs statistical data
print("\n2000s Meteorological Costs Statistical Data: ")
print(meteorological_costs[20:30].describe())

# 2010s meteorological costs statistical data
print("\n2010s Meteorological Costs Statistical Data: ")
print(meteorological_costs[30:40].describe())

print("\n")

# meteorological costs statistical changes 1980s-2010s
meteorological_mean_costs_1980s = meteorological_costs[0:10].mean()
meteorological_mean_costs_2010s = meteorological_costs[30:40].mean()
mean_costs_percent_change_meteorological_1980_to_2020 = 100*(meteorological_mean_costs_2010s-meteorological_mean_costs_1980s)/meteorological_mean_costs_1980s
print("Mean Meteorological Disaster Costs Change (1980s-2010s): {:.2f}%".format(mean_costs_percent_change_meteorological_1980_to_2020))

meteorological_max_costs_1980s = meteorological_costs[0:10].max()
meteorological_max_costs_2010s = meteorological_costs[30:40].max()
max_costs_percent_change_meteorological_1980_to_2020 = 100*(meteorological_max_costs_2010s-meteorological_max_costs_1980s)/meteorological_max_costs_1980s
print("Max Meteorological Disaster Costs Change (1980s-2010s): {:.2f}%".format(max_costs_percent_change_meteorological_1980_to_2020))

meteorological_min_costs_1980s = meteorological_costs[0:10].min()
meteorological_min_costs_2010s = meteorological_costs[30:40].min()
min_costs_percent_change_meteorological_1980_to_2020 = 100*(meteorological_min_costs_2010s-meteorological_min_costs_1980s)/meteorological_min_costs_1980s
print("Min Meteorological Disaster Costs Change (1980s-2010s): {:.2f}%".format(min_costs_percent_change_meteorological_1980_to_2020))

meteorological_std_costs_1980s = meteorological_costs[0:10].std()
meteorological_std_costs_2010s = meteorological_costs[30:40].std()
std_costs_percent_change_meteorological_1980_to_2020 = 100*(meteorological_std_costs_2010s-meteorological_std_costs_1980s)/meteorological_std_costs_1980s
print("Standard Deviation of Meteorological Disaster Costs Change (1980s-2010s): {:.2f}%\n".format(std_costs_percent_change_meteorological_1980_to_2020))

In [ ]:
# HYDROLOGICAL DISASTER COST ANALYSIS

print("HYDROLOGICAL DISASTER COST ANALYSIS")

# hydrological costs statistical data
print("\nTotal Hydrological Costs Statistical Data: ")
print(hydrological_costs.describe())

# 1980s hydrological costs statistical data
print("\n1980s Hydrological Costs Statistical Data: ")
print(hydrological_costs[0:10].describe())

# 1990s hydrological costs statistical data
print("\n1990s Hydrological Costs Statistical Data: ")
print(hydrological_costs[10:20].describe())

# 2000s hydrological costs statistical data
print("\n2000s Hydrological Costs Statistical Data: ")
print(hydrological_costs[20:30].describe())

# 2010s hydrological costs statistical data
print("\n2010s Hydrological Costs Statistical Data: ")
print(hydrological_costs[30:40].describe())

print("\n")

# hydrological costs statistical changes 1980s-2010s
hydrological_mean_costs_1980s = hydrological_costs[0:10].mean()
hydrological_mean_costs_2010s = hydrological_costs[30:40].mean()
mean_costs_percent_change_hydrological_1980_to_2020 = 100*(hydrological_mean_costs_2010s-hydrological_mean_costs_1980s)/hydrological_mean_costs_1980s
print("Mean Hydrological Disaster Costs Change (1980s-2010s): {:.2f}%".format(mean_costs_percent_change_hydrological_1980_to_2020))

hydrological_max_costs_1980s = hydrological_costs[0:10].max()
hydrological_max_costs_2010s = hydrological_costs[30:40].max()
max_costs_percent_change_hydrological_1980_to_2020 = 100*(hydrological_max_costs_2010s-hydrological_max_costs_1980s)/hydrological_max_costs_1980s
print("Max Hydrological Disaster Costs Change (1980s-2010s): {:.2f}%".format(max_costs_percent_change_hydrological_1980_to_2020))

hydrological_min_costs_1980s = hydrological_costs[0:10].min()
hydrological_min_costs_2010s = hydrological_costs[30:40].min()
min_costs_percent_change_hydrological_1980_to_2020 = 100*(hydrological_min_costs_2010s-hydrological_min_costs_1980s)/hydrological_min_costs_1980s
print("Min Hydrological Disaster Costs Change (1980s-2010s): {:.2f}%".format(min_costs_percent_change_hydrological_1980_to_2020))

hydrological_std_costs_1980s = hydrological_costs[0:10].std()
hydrological_std_costs_2010s = hydrological_costs[30:40].std()
std_costs_percent_change_hydrological_1980_to_2020 = 100*(hydrological_std_costs_2010s-hydrological_std_costs_1980s)/hydrological_std_costs_1980s
print("Standard Deviation of Hydrological Disaster Costs Change (1980s-2010s): {:.2f}%\n".format(std_costs_percent_change_hydrological_1980_to_2020))